In [3]:
import pandas as pd
import torch
import mysql.connector
import dgl.data
import numpy as np
from dgl import save_graphs, load_graphs
import torch as th
import gensim
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import requests
import time

[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
cnx = mysql.connector.connect(user='david', password='',
                          host='127.0.0.1',
                          database='computervision')
cursor = cnx.cursor()
headers = {"x-api-key": ""}

DatabaseError: 2003 (HY000): Can't connect to MySQL server on '127.0.0.1:3306' (111)

In [4]:
test_papers = []
with open(r'./test_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        test_papers.append(x)
in_params_test = ','.join(['%s'] * len(test_papers))

train_papers = []
with open(r'./train_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        train_papers.append(x)

total_papers = train_papers + test_papers
in_params_train = ','.join(['%s'] * len(train_papers))

val_papers = []
with open(r'./val_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        val_papers.append(x)

in_params_val = ','.join(['%s'] * len(val_papers))
in_params = ','.join(['%s'] * len(train_papers + test_papers))

In [5]:
def remap(x, mapping):
    return mapping.get(x, x)

In [14]:
cursor.execute("select distinct p2.PaperID from Papers p, authoredBy b, authoredBy b2, Papers p2 where p.PaperID in (%s) and p.PaperID = b.PaperID and b.AuthoredByID = b2.AuthoredByID and b2.PaperID = p2.PaperID and p2.Pub_Year != 2022" % in_params_test, test_papers)
train_papers = [x[0] for x in cursor.fetchall()]
total_papers = train_papers + test_papers
in_params = ','.join(['%s'] * len(total_papers))

In [15]:
cursor.execute("select b.ReferenceID, b.ReferencedByID from referencedBy b, Papers p where b.ReferencedByID in (%s) and b.ReferenceID = p.PaperID and p.`primary author` in (select AuthorID from affiliatedTo) and b.ReferenceID in (select PaperID from Papers) and b.ReferencedByID in (select PaperID from Papers)" % in_params, total_papers)
paper_edges = np.array(cursor.fetchall())
all_papers = np.unique(paper_edges)

remapped_papers = {all_papers[i]: i  for i in range(len(all_papers))}
vec_remap = np.vectorize(remap, otypes=[str])
paper_edges = vec_remap(paper_edges, remapped_papers)

all_papers_list = all_papers.tolist()
all_params = ','.join(['%s'] * len(all_papers_list))

#cursor.execute("select p.PaperID, p.`primary author`, a.Gender, p.Title from Papers p, Authors a where p.PaperID in (%s) and p.`primary author` in (select AuthorID from affiliatedTo) and p.`primary author` = a.AuthorID" % all_params, all_papers_list)
cursor.execute("select p.PaperID, a.AuthorID, a.Gender, p.Title from Papers p, Authors a, authoredBy ab where p.PaperID in (%s) and p.PaperID = ab.PaperID and ab.AuthoredByID = a.AuthorID" % all_params, all_papers_list) #and a.AuthorID in (select AuthorID from affiliatedTo)
author_edges = np.array(cursor.fetchall())

all_authors = np.unique(author_edges[:,1])
remapped_authors = {all_authors[i]: i  for i in range(len(all_authors))}

author_edges[:,1] = vec_remap(author_edges[:,1], remapped_authors)
author_edges[:,0] = vec_remap(author_edges[:,0], remapped_papers)

In [6]:
model_path = './graphs/GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
def title_to_vec(title):
    title = title.replace("-", " ")
    tokens = nltk.word_tokenize(title)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    vectors = []
    for word in tokens:
        try:
            vec = model[word.lower()]
            vectors.append(vec)
        except KeyError:
            #print(word)
            continue
    title_vector = np.zeros(300)
    try:
        title_vector = sum(vectors) / len(vectors)
    except ZeroDivisionError:
        pass
    return title_vector

In [8]:
all_papers_info = pd.DataFrame(author_edges)
all_papers_info[0] = all_papers_info[0].astype(int)
all_papers_info = all_papers_info.sort_values(0)
paper_feats = []
stop_words = set(stopwords.words('english'))
for title in all_papers_info[3]:
    title_vec = title_to_vec(title)
    paper_feats.append(title_vec)
paper_feats = torch.tensor(paper_feats)

NameError: name 'author_edges' is not defined

In [10]:
def title_to_vec(title):
    title = title.replace("-", " ")
    tokens = nltk.word_tokenize(title)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    vectors = []
    for word in tokens:
        try:
            vec = model[word.lower()]
            vectors.append(vec)
        except KeyError:
            #print(word)
            continue
    title_vector = np.zeros(300)
    try:
        title_vector = sum(vectors) / len(vectors)
    except ZeroDivisionError:
        pass
    return title_vector

In [11]:
def request_author_information(authors):
    resp = requests.post("https://api.semanticscholar.org/graph/v1/author/batch?fields=papers.title,affiliations,paperCount,citationCount,hIndex", headers=headers, json={"ids": authors}).json()
    return resp

def request_author_individual_dfinformation(author):
    resp = requests.get(f"https://api.semanticscholar.org/graph/v1/author/{author}?fields=papers.title,paperCount,citationCount,hIndex", headers=headers).json()
    return resp

def convert_history_to_vector(response):
    if not response:
        author_hist = np.zeros(300)
        author_hist = np.append(author_hist, [0,0,0])
        return author_hist

    author_papers = response["papers"]
    author_hist = []

    for paper in author_papers:
        paperID = paper["paperId"]
        if paperID in test_papers:
            continue
        title = paper["title"]
        title_vect = title_to_vec(title)
        if np.count_nonzero(title_vect) > 0:
            author_hist.append(title_vect)
    if len(author_hist) != 0:
        author_hist = np.mean(np.array(author_hist), axis=0)
    else:
        author_hist = np.zeros(300)

    author_paperCount = response["paperCount"]
    author_citationCount = response["citationCount"]
    author_hIndex = response["hIndex"]
    author_hist = np.append(author_hist, [author_paperCount, author_citationCount, author_hIndex])

    return author_hist

In [40]:
np.savetxt('all_authors.txt', all_authors.astype(int))

In [12]:
all_authors = np.loadtxt('./graphs/all_authors.txt', dtype=int)

/tmp/ipykernel_2864/294877418.py:1: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  all_authors = np.loadtxt('./graphs/all_authors.txt', dtype=int)


In [13]:
#author_bug = request_author_individual_dfinformation("47399096")
#all_authors = np.delete(all_authors, 101791)
all_authors = np.loadtxt('./graphs/all_authors.txt', dtype=int)

In [14]:
index = 0
start = 100
author_feats = []
while start*index <= len(all_authors):
    print(f"{start*index}/{len(all_authors)}")
    batch_authors = all_authors[start*index: (start*index)+start]
    index += 1

    batch_process = True

    response_batch = request_author_information(batch_authors.tolist())
    if "message" in response_batch:
        batch_process = False

    if batch_process:
        for response in response_batch:
            author_hist = convert_history_to_vector(response)
            author_feats.append(author_hist)
            if author_hist.shape[0] != 303:
                print(response)
                print(author_hist.shape)
    else:
        for author_id in batch_authors:
            author_information = request_author_individual_dfinformation(author_id)
            author_hist_vec = convert_history_to_vector(author_information)
            author_feats.append(author_hist_vec)

0/116740
100/116740
200/116740
300/116740
400/116740
500/116740
600/116740
700/116740
800/116740
900/116740
1000/116740
1100/116740
1200/116740
1300/116740
1400/116740
1500/116740
1600/116740
1700/116740
1800/116740
1900/116740
2000/116740
2100/116740
2200/116740
2300/116740
2400/116740
2500/116740
2600/116740
2700/116740
2800/116740
2900/116740
3000/116740
3100/116740
3200/116740
3300/116740
3400/116740
3500/116740
3600/116740
3700/116740
3800/116740
3900/116740
4000/116740
4100/116740
4200/116740
4300/116740
4400/116740
4500/116740
4600/116740
4700/116740
4800/116740
4900/116740
5000/116740
5100/116740
5200/116740
5300/116740
5400/116740
5500/116740
5600/116740
5700/116740
5800/116740
5900/116740
6000/116740
6100/116740
6200/116740
6300/116740
6400/116740
6500/116740
6600/116740
6700/116740
6800/116740
6900/116740
7000/116740
7100/116740
7200/116740
7300/116740
7400/116740
7500/116740
7600/116740
7700/116740
7800/116740
7900/116740
8000/116740
8100/116740
8200/116740
8300/116740
8400